In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
import pickle

In [2]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Preprocess the data
# Dropping Irrelevant Features

data = data.drop(["RowNumber","CustomerId","Surname"],axis=1)

In [4]:
# Encoding Categorical Variables 
# Encoding Gender using LabelEncoder

encoder_gender = LabelEncoder()
data["Gender"] = encoder_gender.fit_transform(data["Gender"])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [5]:
# Encoding Geography Feature using OneHotEncoder

onehot_encoder_geography = OneHotEncoder(sparse_output=False)
encoder_geography = onehot_encoder_geography.fit_transform(data[["Geography"]])
geo_encoded_df = pd.DataFrame(encoder_geography,columns=onehot_encoder_geography.get_feature_names_out(["Geography"]))
data = pd.concat([data.drop("Geography",axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [8]:
# Splitting into Dependent and Independent Features
X = data.drop("EstimatedSalary",axis=1)
y = data["EstimatedSalary"]

# Split into training and testing sets
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# Scale down these features 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
# Saving Encoder and Scaler
with open("encoder_gender.pkl","wb") as file:
    pickle.dump(encoder_gender,file)

with open("onehot_encoder_geography.pkl","wb") as file:
    pickle.dump(onehot_encoder_geography,file)

with open("scaler.pkl","wb") as file:
    pickle.dump(scaler,file)

# ANN Implementation

### Steps in ANN Implementation

1. **Sequential Network**  
   Build the model using a sequential architecture.

2. **Dense - Hidden Neuron Creation**  
   Add dense layers to create hidden neurons.

3. **Activation Function**  
   Choose activation functions such as `sigmoid`, `tanh`, `relu`, or `leakyrelu`.

4. **Optimizer - Backpropagation**  
   Select an optimizer (e.g., Adam, SGD) responsible for updating the weights during backpropagation.

5. **Loss Function**  
   Define the loss function to measure model error.

6. **Metrics**  
   - Classification: Accuracy  
   - Regression: MSE (Mean Squared Error), MAE (Mean Absolute Error)

7. **Training & Logging**  
   Train the model and log results using TensorBoard for visualization.

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense

In [11]:
# Build ANN Model

input_dim = X_train.shape[1]
model = Sequential([
    Input(shape=(input_dim,)),
    Dense(64,activation="relu"), # Hidden Layer 1 - connected with input layer
    Dense(32,activation="relu"),  # Hidden Layer 2
    Dense(1)  # Output Layer for Regression -> Default Activation Function is applied - Linear Activation Function
])

2025-09-17 18:27:08.734824: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-09-17 18:27:08.735025: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-09-17 18:27:08.735041: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-09-17 18:27:08.735851: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-17 18:27:08.737189: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [12]:
# Forward and Backward Propogation

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="mean_absolute_error",
    metrics=["mae"]
    )

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

# Set up Tensorboard

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [14]:
# Set up Early Stopping 

early_stopping_callback = EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)

In [15]:
# Train the model

history = model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


2025-09-17 18:32:00.643550: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 100427.1797 - mae: 100427.1797 - val_loss: 98711.4062 - val_mae: 98711.4062
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 100388.3203 - mae: 100388.3203 - val_loss: 98631.3203 - val_mae: 98631.3203
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 100259.7422 - mae: 100259.7422 - val_loss: 98412.3594 - val_mae: 98412.3594
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 99974.3828 - mae: 99974.3828 - val_loss: 97993.5156 - val_mae: 97993.5156
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 99463.0312 - mae: 99463.0312 - val_loss: 97271.3438 - val_mae: 97271.3438
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 98606.7109 - mae: 98606.7109 - val_loss: 96182.4219 - val_mae: 96182.4219
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 97444.4375 - mae: 97444.4375 - val_loss: 94855.6484 - val_mae: 94855.6484
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 

In [16]:
model.save("model.h5")

In [18]:
%load_ext tensorboard
%tensorboard --logdir logs/fit

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 91081), started 0:00:27 ago. (Use '!kill 91081' to kill it.)

In [ ]:
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers

def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Input(shape=(X_train.shape[1],)))
    
    # Tune number of hidden layers
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(layers.Dense(
            units=hp.Int(f'units_{i}', min_value=8, max_value=128, step=8),
            activation='relu'
        ))
    
    model.add(layers.Dense(1))  # Output layer (for regression)

    model.compile(optimizer='adam', loss='mae', metrics=['mae'])
    return model

# Run tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_mae',
    max_trials=10,
    executions_per_trial=2,
    directory='tuner_dir',
    project_name='salary_prediction'
)

tuner.search(X_train, y_train, epochs=30, validation_split=0.2)
best_model = tuner.get_best_models(num_models=1)[0]

Trial 10 Complete [00h 01m 32s]
val_mae: 78929.71875

Best val_mae So Far: 49788.908203125
Total elapsed time: 00h 17m 16s


/Users/lakshaysindhu/Desktop/GEN-AI/NLP/nenv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [24]:
print(best_model)

<Sequential name=sequential, built=True>


In [19]:
# Evaluate the model on test data

test_loss,test_mae = model.evaluate(X_test,y_test)
print(f"Test MAE : {test_mae}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 87496.3281 - mae: 87496.3281
Test MAE : 87496.328125


In [20]:
model.save("regression_model.h5")